<a href="https://colab.research.google.com/github/sandstorm12/RubikaInterviewExam/blob/dev/rubika_interview_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Issues and future work

1. Try oversampling using augmentation
2. Try not to use Keras Applications pre-processing for the custom model :)

In [ ]:
#@title System check { form-width: "20%" }

!nvidia-smi

In [ ]:
#@title Installation { form-width: "20%" }

!pip3 install focal-loss

In [ ]:
#@title Imports { form-width: "20%"}

import sklearn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from focal_loss import SparseCategoricalFocalLoss

In [ ]:
#@title Load dataset { form-width: "20%" }

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print("x_train: {} y_train: {} x_test: {} y_test: {}".format(
        x_train.shape, y_train.shape, x_test.shape, y_test.shape
    )
)

print("x_train --> max: {} mean: {} min: {}".format(
        np.max(x_train), np.mean(x_train), np.min(x_train)
    )
)

print("y_train --> unique_values: {}".format(
        np.unique(y_train)
    )
)

In [ ]:
#@title Preprocessing { form-width: "20%" }

x_train_preprocessed = tf.keras.applications.densenet.preprocess_input(x_train)
x_test_preprocessed = tf.keras.applications.densenet.preprocess_input(x_test)

print("x_train --> max: {} mean: {} min: {}".format(
        np.max(x_train_preprocessed),
        np.mean(x_train_preprocessed),
        np.min(x_train_preprocessed)
    )
)

In [ ]:
#@title One class separation { form-width: "20%" }

selected_class =  8#@param {type:"integer"}

y_train_one_class = y_train.copy()
y_train_one_class[y_train == selected_class] = 0
y_train_one_class[y_train != selected_class] = 1

y_test_one_class = y_test.copy()
y_test_one_class[y_test == selected_class] = 0
y_test_one_class[y_test != selected_class] = 1

print(np.unique(y_train_one_class))
print(np.unique(y_test_one_class))

In [ ]:
#@title Build model (transfer learning) { form-width: "20%" }

focal_loss = False #@param {type:"boolean"}


model = tf.keras.Sequential()

model.add(
    tf.keras.applications.DenseNet121(
        include_top=False,
        weights="imagenet",
        input_shape=(32, 32, 3),
    )
)

for layer in model.layers:
    layer.trainable = False

model.add(
    tf.keras.layers.Flatten()
)

model.add(
    tf.keras.layers.Dense(512, activation="relu")
)

model.add(
    tf.keras.layers.Dense(128, activation="relu")
)

model.add(
    tf.keras.layers.Dense(2, activation="softmax")
)

loss_func = "sparse_categorical_crossentropy"
if focal_loss:
    loss_func = SparseCategoricalFocalLoss(gamma=2)


model.compile(
    loss=loss_func,
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

In [ ]:
#@title Build model (from scratch) { form-width: "20%" }

focal_loss = True #@param {type:"boolean"}
first_layer_filters = 32 #@param {type:"integer"}
second_layer_filters = 64 #@param {type:"integer"}
third_layer_filters = 128 #@param {type:"integer"}


model = tf.keras.Sequential()

model.add(
    tf.keras.layers.InputLayer((32, 32, 3))
)

model.add(
    tf.keras.layers.Conv2D(first_layer_filters, (3, 3), activation="relu")
)
model.add(
    tf.keras.layers.Conv2D(first_layer_filters, (3, 3), activation="relu")
)
model.add(
    tf.keras.layers.MaxPool2D((2, 2))
)
model.add(
    tf.keras.layers.BatchNormalization()
)

model.add(
    tf.keras.layers.Conv2D(second_layer_filters, (3, 3), activation="relu")
)
model.add(
    tf.keras.layers.Conv2D(second_layer_filters, (3, 3), activation="relu")
)
model.add(
    tf.keras.layers.MaxPool2D((2, 2))
)
model.add(
    tf.keras.layers.BatchNormalization()
)

model.add(
    tf.keras.layers.Conv2D(third_layer_filters, (3, 3), activation="relu")
)
model.add(
    tf.keras.layers.Conv2D(third_layer_filters, (3, 3), activation="relu")
)

model.add(
    tf.keras.layers.Flatten()
)
model.add(
    tf.keras.layers.BatchNormalization()
)

model.add(
    tf.keras.layers.Dense(2, activation="softmax")
)

loss_func = "sparse_categorical_crossentropy"
if focal_loss:
    loss_func = SparseCategoricalFocalLoss(gamma=2)

model.compile(
    loss=loss_func,
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

In [ ]:
#@title Train { form-width: "20%" }
class_weight = False #@param {type:"boolean"}
save_on_each_epoch = True #@param {type:"boolean"}
epochs =  50 #@param {type:"integer"}
batch_size = 32 #@param {type:"integer"}

weights = None
if class_weight:
    weights = sklearn.utils.class_weight.compute_class_weight(
        "balanced",
        np.unique(y_train_one_class),
        y_train_one_class.ravel()
    )
    weights = {0: weights[0], 1: weights[1]}
    print("Class weights: {}".format(weights))
    

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print("epoch", epoch)
        y_pred = np.argmax(model.predict(x_test_preprocessed), axis=1)
        print(classification_report(y_test_one_class, y_pred))

        if save_on_each_epoch:
            self.model.save("model_epoch_{}.hd5".format(epoch))


model.fit(
    x_train_preprocessed, y_train_one_class,
    validation_split=.1, batch_size=batch_size, epochs=epochs,
    callbacks=[CustomCallback()], class_weight=weights
)

In [ ]:
#@title Evaluation { form-width: "20%" }

load = True #@param {type:"boolean"}
model_epoch =  17 #@param {type:"integer"}

if load:
    model = tf.keras.models.load_model(
        "model_epoch_{}.hd5".format(model_epoch)
    )

y_pred = np.argmax(model.predict(x_test_preprocessed), axis=1)


print("Classification report: \n{}".format(
        classification_report(y_test_one_class, y_pred)
    )
)

print("Confusion matrix: \n{}".format(
        confusion_matrix(y_test_one_class, y_pred)
    )
)

In [ ]:
#@title Feature extraction { form-width: "20%" }

topless_model = tf.keras.Model(
    inputs=model.input,
    outputs=model.get_layer(model.layers[-2].name).output
)

features = topless_model.predict(x_test_preprocessed)

print("Features shape: {}".format(features.shape))

In [ ]:
#@title Dimensionality reduction using TSN-e { form-width: "20%" }

features_2d= TSNE(n_components=2).fit_transform(features)

print("New features shape: {}".format(features_2d.shape))

In [ ]:
#@title Feature visualization { form-width: "20%" }

fig, ax = plt.subplots()
ax.scatter(
    features_2d[(y_test_one_class==1).ravel(), 0],
    features_2d[(y_test_one_class==1).ravel(), 1],
    label="Other classes"
)
ax.scatter(
    features_2d[(y_test_one_class==0).ravel(), 0],
    features_2d[(y_test_one_class==0).ravel(), 1],
    label="Selected class"
)
fig.legend()
fig.show()

fig.savefig('plot.png', format='png', dpi=1200)